In [1]:
# Exercises for chapter two of Hands-On Machine Learning


In [2]:
#Housing dataset

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import os

In [5]:
import sklearn

In [6]:
import seaborn as sb

In [7]:
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)


housing = load_housing_data()

In [8]:
#explore the data

In [9]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [10]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [11]:
#mising values for total_bedrooms 
#one categorical feature
#features need to be combined to be useful: total_bedrooms, total_rooms, households

In [12]:
#make training and test sets
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [13]:
#stratified sampling for median income
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [14]:
strat_train_set.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'income_cat'],
      dtype='object')

In [15]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [16]:
strat_train_set.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [17]:
housing = strat_train_set.copy()

In [18]:
# experimenting with some combinations
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [19]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy() # start from clean dataset

In [20]:
#fill in missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

housing_num = housing.drop("ocean_proximity", axis=1)

imputer.fit(housing_num)

X = imputer.transform(housing_num) # can use this trained inputer in the future!!!!

# put back into pandas
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

In [21]:
#process categorical feature
housing_cat = housing[["ocean_proximity"]]

from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [ ]:
# Make custom transformers

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin

In [26]:
#number/index the columns
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

In [28]:
# create a class object and implement methods fit(), transform(), fit_transform()
#will return rooms_per_household and population_per_household
#optionally will return bedrooms_per_household
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    #executed when this class is initiated. Doesnt need to be called by user. 
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        #user sets this parameter to True or False
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        #combine attributes to get a new one (average rooms per house hold)
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        #second new feature
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: 
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [30]:
#initialize and utilize the class object
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [31]:
# convert to a dataframe
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
17606,-121.89,37.29,38,1568,351,710,339,2.7042,<1H OCEAN,4.62537,2.0944
18632,-121.93,37.05,14,679,108,306,113,6.4214,<1H OCEAN,6.00885,2.70796
14650,-117.2,32.77,31,1952,471,936,462,2.8621,NEAR OCEAN,4.22511,2.02597
3230,-119.61,36.31,25,1847,371,1460,353,1.8839,INLAND,5.23229,4.13598
3555,-118.59,34.23,17,6592,1525,4459,1463,3.0347,<1H OCEAN,4.50581,3.04785


# preprocess numerical attributes

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [33]:
#utilize the pipeline functionality

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), #impute the median value to missing values
        ('attribs_adder', CombinedAttributesAdder()), #combine the rooms attributes via class object
        ('std_scaler', StandardScaler()), #scale the attributes
    ])

In [34]:
#run pipeline
housing_num_tr = num_pipeline.fit_transform(housing_num)

(16512, 8)

In [35]:
from sklearn.compose import ColumnTransformer

In [38]:
#get full lists of new attributes
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

#create a pipeline which uses previous pipeline
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

#generate a new array with preprocessed cat and num attributes using the pipelines and class object
housing_prepared = full_pipeline.fit_transform(housing)

In [39]:
(housing_prepared.shape,housing.shape)

((16512, 16), (16512, 9))

In [40]:
housing_labels.shape

(16512,)

In [106]:
housing_num.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')

# Exercises 



# 1 Try an SVM with various hyperparameters for kernel (linear vs rbf), C, gamma. How does the best SVR perform? 

In [43]:
#load metric: RMSE MAE
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn import svm

In [ ]:
#SV_Reg = svm.SVR()
#SV_Reg.fit(housing_prepared, housing_labels, kernel='rbf', C=.1.0, gamma='scale')

In [ ]:
#SV_Reg = svm.SVR()
#SV_Reg.fit(housing_prepared, housing_labels, kernel='linear', C=1.0')

In [44]:
from sklearn.model_selection import GridSearchCV

In [2]:
#create a parameter grid to use in grid search
param_grid= [
    {'kernel':['linear'], 'C':[0.1, 0.5, 1.0, 2.0, 5]},
    {'kernel':['rbf'], 'C':[0.1, 0.5, 1.0, 2.0, 5], 'gamma':['scale', 'auto']}
]

In [50]:
#initialize SVR
SV_Reg = svm.SVR()

#initialize the grid search
grid_search = GridSearchCV(SV_Reg, param_grid, cv=5,
                        scoring='neg_mean_squared_error',
                        return_train_score=True)

#conduct grid search
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [0.1, 0.5, 1.0, 2.0, 5], 'kernel': ['linear']},
                         {'C': [0.1, 0.5, 1.0, 2.0, 5],
                          'gamma': ['scale', 'auto'], 'kernel': ['rbf']}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [51]:
#return best parameters
grid_search.best_params_

{'C': 5, 'kernel': 'linear'}

In [52]:
#the model is not very good...
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

118248.96638734567 {'C': 0.1, 'kernel': 'linear'}
115595.79530238075 {'C': 0.5, 'kernel': 'linear'}
112571.9845974018 {'C': 1.0, 'kernel': 'linear'}
107136.26398576611 {'C': 2.0, 'kernel': 'linear'}
95066.89952171071 {'C': 5, 'kernel': 'linear'}
118892.47248281227 {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
118892.81131075782 {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
118773.19576051271 {'C': 0.5, 'gamma': 'scale', 'kernel': 'rbf'}
118775.65803868763 {'C': 0.5, 'gamma': 'auto', 'kernel': 'rbf'}
118631.71004256308 {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}
118634.4106689143 {'C': 1.0, 'gamma': 'auto', 'kernel': 'rbf'}
118352.3307726622 {'C': 2.0, 'gamma': 'scale', 'kernel': 'rbf'}
118356.69930471006 {'C': 2.0, 'gamma': 'auto', 'kernel': 'rbf'}
117508.07539783238 {'C': 5, 'gamma': 'scale', 'kernel': 'rbf'}
117531.43159173557 {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}


In [53]:
#re-try grid search but look for c values > 5
param_grid= [
    {'kernel':['linear'], 'C':[5, 15, 50, 100, ]},
]

#initialize SVR
SV_Reg = svm.SVR()

#initialize the grid search
grid_search = GridSearchCV(SV_Reg, param_grid, cv=5,
                        scoring='neg_mean_squared_error',
                        return_train_score=True)

#conduct grid search
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [5, 15, 50, 100], 'kernel': ['linear']}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [54]:
grid_search.best_params_

{'C': 100, 'kernel': 'linear'}

In [55]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

95066.89952171071 {'C': 5, 'kernel': 'linear'}
80588.78101452661 {'C': 15, 'kernel': 'linear'}
73235.12213707059 {'C': 50, 'kernel': 'linear'}
71603.17710755806 {'C': 100, 'kernel': 'linear'}


In [62]:
#re-try grid search but look for c values > 100
param_grid= [
    {'kernel':['linear'], 'C':[150000, 300000, 750000]},
]

#initialize SVR
SV_Reg = svm.SVR()

#initialize the grid search
grid_search = GridSearchCV(SV_Reg, param_grid, cv=5,
                        scoring='neg_mean_squared_error',
                        return_train_score=True)

#conduct grid search
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [150000, 300000, 750000], 'kernel': ['linear']}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [60]:
#ran the grid search multiple times because my estimates for a good C value were just way off
grid_search.best_params_

{'C': 75000, 'kernel': 'linear'}

In [61]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

71603.17710755806 {'C': 100, 'kernel': 'linear'}
70445.3835347414 {'C': 1000, 'kernel': 'linear'}
70374.76833985467 {'C': 10000, 'kernel': 'linear'}
70355.15376708687 {'C': 75000, 'kernel': 'linear'}


In [63]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

70354.52059373276 {'C': 150000, 'kernel': 'linear'}
70356.16326032735 {'C': 300000, 'kernel': 'linear'}
70356.55134965181 {'C': 750000, 'kernel': 'linear'}


In [64]:
#optimal settings seem to be a linear kernel with C=150000

In [ ]:
#How does the SVR predictor perform?
#not as well as the linear regression

# Exercise 2 

In [ ]:
#Try replacing grid search with RandomizedSearchCV

In [69]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [87]:
#re-try grid search but look for c values > 100
param_grid= [
    {'C':randint(50000,500000)},
    {'kernel':['linear']}
]

#initialize SVR
SV_Reg = svm.SVR()

#initialize the grid search
rand_search = RandomizedSearchCV(SV_Reg, param_grid, cv=5, n_iter = 8,
                        scoring='neg_mean_squared_error',
                        return_train_score=True)

#conduct grid search
rand_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, estimator=SVR(), n_iter=8,
                   param_distributions=[{'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000014CF0870C40>},
                                        {'kernel': ['linear']}],
                   return_train_score=True, scoring='neg_mean_squared_error')

In [79]:
#this was actually using an rbf kernel
cvres = rand_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

56594.407206242446 {'C': 185392}
56351.75220782828 {'C': 236369}
56679.065248512314 {'C': 172522}
56712.37112893572 {'C': 167307}
55995.878275917894 {'C': 341457}
55775.67890545486 {'C': 440087}
55732.90781532693 {'C': 464738}
55841.286903109074 {'C': 401613}


In [80]:
rand_search.best_params_

{'C': 464738}

In [88]:
#reran rand_search with a linear kernel 
cvres = rand_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

112571.9845974018 {'kernel': 'linear'}
55898.09020897089 {'C': 376424}
112571.9845974018 {'kernel': 'linear'}
112571.9845974018 {'kernel': 'linear'}
112571.9845974018 {'kernel': 'linear'}
112571.9845974018 {'kernel': 'linear'}
112571.9845974018 {'kernel': 'linear'}
56088.98362973389 {'C': 305582}


In [89]:
rand_search.best_params_

{'C': 376424}

In [72]:
#using random search after grid search yielded a far better hyperparameter combo. 
#but I may or may not have overfit the data

# Exercise 3 and 4

In [81]:
#try adding a transformer to the preparation pipeline to select only the most important attributes.
#Try creating a full pipeline that does the data preparation plus the prediction

# Use grid search combined with a custom transformer to determine if the bedrooms_per_room is useful or not
# treat it as a hyperparameter within CombindeAttributesAdder transformer. 
# for some predictors grid search can tell you the most important attributes using 
#       grid_search.best_estimator_.feature_importances_
#       however svr has no such feature

In [ ]:
# feature importance from a linear regression show that of the categorical variables only the one indicating INLAND
# is useful. 

[(0.358144658512645, 'median_income'),
 (0.16793321295702934, 'INLAND'),
 (0.1069647184208425, 'pop_per_hhold'),
 (0.0738136258600976, 'longitude'),
 (0.06495039142276503, 'latitude'),
 (0.056819255680012865, 'bedrooms_per_room'),
 (0.05614739162339144, 'rooms_per_hhold'),
 (0.04085123706927927, 'housing_median_age'),
 (0.014731355618160519, 'households'),
 (0.014375075462630688, 'population'),
 (0.014242335237696264, 'total_rooms'),
 (0.013788082754377505, 'total_bedrooms'),
 (0.012219505724080364, '<1H OCEAN'),
 (0.0030419821094427573, 'NEAR OCEAN'),
 (0.0019223129694366796, 'NEAR BAY'),
 (5.485857811232326e-05, 'ISLAND')]

In [108]:
#number/index the columns
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

In [171]:
# create a class object and implement methods fit(), transform(), fit_transform()
#will return rooms_per_household and population_per_household
#optionally will return bedrooms_per_household
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    #executed when this class is initiated. Doesnt need to be called by user. 
    def __init__(self, add_bedrooms_per_room = True, drop_old_attb=True): # no *args or **kargs
        #user sets this parameter to True or False
        self.add_bedrooms_per_room = add_bedrooms_per_room
        self.drop_old_attb = drop_old_attb
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        #combine attributes to get a new one (average rooms per house hold)
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        #second new feature
        population_per_household = X[:, population_ix] / X[:, households_ix]
        #make attribute combinations
        if self.add_bedrooms_per_room: 
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            X = np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            X = np.c_[X, rooms_per_household, population_per_household]
        if self.drop_old_attb:
            X = np.delete(X, [rooms_ix, bedrooms_ix, population_ix, households_ix], 1)
        return X

In [185]:
#ok so I can reuse the numerical pipeline num_pipeline since we want to cut just cat variabls
# need to add a bit to the existing custom transformer which will strip away all the categorical 
# variables except INLAND. Also want to drop total_rooms, total_bedrooms, population, and households 
# since CombinedAttributesAdder will add combined versions of those. 

class CatAttributesRemover(BaseEstimator, TransformerMixin):
    def __init__(self, drop_most_cats=True): # no *args or **kargs
        self.drop_most_cats = drop_most_cats
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        if self.drop_most_cats:
            X = (X=='INLAND').astype(int)
            return X
        

In [186]:
#get full lists of attributes
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

#repasting num_pipeline here so I can set additional attribs =True
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), #impute the median value to missing values
        ('attribs_adder', CombinedAttributesAdder(add_bedrooms_per_room=True, drop_old_attb=True)), #combine the rooms attributes via class object
        ('std_scaler', StandardScaler()), #scale the attributes
    ])

#categorical pipeline
cat_pipeline = Pipeline([
    #('one_hot', OneHotEncoder()),
    ('drop', CatAttributesRemover(drop_most_cats=True))
])

#create a pipeline which uses num_pipeline
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        #("cat", OneHotEncoder(), cat_attribs),
        ('cat', cat_pipeline, cat_attribs),
    ])

#generate a new array with preprocessed cat and num attributes using the pipelines and class object
housing_prepared = full_pipeline.fit_transform(housing)

In [187]:
housing_prepared.shape

(16512, 8)

In [188]:
print(type(housing_prepared))
print(housing_prepared.shape)
print(list(housing.columns))
list(housing.columns[0:3]) + list(housing.columns[7:]) + list(["rooms_per_household", "population_per_household", "bedrooms_per_room"])

<class 'numpy.ndarray'>
(16512, 8)
['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity']


['longitude',
 'latitude',
 'housing_median_age',
 'median_income',
 'ocean_proximity',
 'rooms_per_household',
 'population_per_household',
 'bedrooms_per_room']

In [190]:
#need to return column names so we know what is going on
# convert to a dataframe
housing_prepared_df = pd.DataFrame(
    housing_prepared,
    columns=list(housing.columns[0:3]) + ["mediant_income", "rooms_per_household", "population_per_household", "bedrooms_per_room", "INLAND"],
    index=housing.index)
housing_prepared_df.head()

,longitude,latitude,housing_median_age,mediant_income,rooms_per_household,population_per_household,bedrooms_per_room,INLAND
17606,-1.156043,0.771950,0.743331,-0.614937,-0.312055,-0.086499,0.155318,0.0
18632,-1.176025,0.659695,-1.165317,1.336459,0.217683,-0.033534,-0.836289,0.0
14650,1.186849,-1.342183,0.186642,-0.532046,-0.465315,-0.092405,0.422200,0.0
3230,-0.017068,0.313576,-0.290520,-1.045566,-0.079661,0.089736,-0.196453,1.0
3555,0.492474,-0.659299,-0.926736,-0.441437,-0.357834,-0.004194,0.269928,0.0
